# ETL - Steam games

In [43]:
#Se importan las librerías a necesarias.
import pandas as pd
import numpy as np
import json
import pyarrow as pa
import pyarrow.parquet as pq
import gzip

### Extracción de datos

In [44]:
juegos_json = 'C:\\Users\\fedez\\OneDrive\\Escritorio\\PI-MLOps\\Data\\steam_games.json.gz'
#Se genera una lista vacía para almacenar los datos en ella.
juegos = []
with gzip.open(juegos_json, 'rb') as archivo:
    #Se crea un loop para ir incorporando los elementos a la lista.
    for linea in archivo:
        data= json.loads(linea)
        juegos.append(data)

In [45]:
# Se crea un dataframe a partir del contenido del archivo.
df_juegos = pd.DataFrame(juegos)

In [46]:
df_juegos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


### Eliminación de valores nulos

In [47]:
df_juegos

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [48]:
df_juegos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [49]:
#Se eliminan las filas con valores nulos.
juegos_sin_nulos = df_juegos.dropna()

Se crea una nueva columna con los años de lanzamiento de cada juego ya que en los endpoints se pide solo el año.

In [52]:
# Se estandariza la columna a formato "datetime".
juegos_sin_nulos.loc[:, 'release_date'] = pd.to_datetime(juegos_sin_nulos['release_date'], format='mixed', errors='coerce')
# Se extrae el año de lanzamiento del producto y se coloca en la columna "release_year"
juegos_sin_nulos.loc[:, 'release_year'] = juegos_sin_nulos['release_date'].dt.year

### Eliminación de columnas innecesarias.

In [53]:
juegos_sin_nulos.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,release_year
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018.0


In [54]:
# Se procede a eliminar las columnas "publisher", "app_name", "url", "specs", "early_access", "tags", "reviews_url" y "release_date" 
#  ya que no son necesarias para las funciones a desarrollar o poseen valores que ya están presentes en otras columnas.
df_juegos = juegos_sin_nulos.drop(['publisher', 'app_name', 'url', 'specs', 'early_access', 'tags', 'reviews_url','release_date'], axis=1)

In [55]:
df_juegos.head()

,genres,title,price,id,developer,release_year
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,4.99,761140,Kotoshiro,2018.0
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Free To Play,643980,Secret Level SRL,2018.0
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Free to Play,670290,Poolians.com,2017.0
88313,"[Action, Adventure, Casual]",弹炸人2222,0.99,767400,彼岸领域,2017.0
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,3.99,772540,Trickjump Games Ltd,2018.0


### Explotación de la columna "genres".

In [56]:
df_juegos = df_juegos.explode("genres")

In [57]:
df_juegos.head(6)

,genres,title,price,id,developer,release_year
88310,Action,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018.0
88310,Casual,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018.0
88310,Indie,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018.0
88310,Simulation,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018.0
88310,Strategy,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018.0
88311,Free to Play,Ironbound,Free To Play,643980,Secret Level SRL,2018.0


### Eliminación de valores duplicados.

In [58]:
# Se eliminan los elementos duplicados existentes entre las columnas "title" y "genres".
df_juegos = df_juegos.drop_duplicates(subset=['title', 'genres'])

In [59]:
df_juegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55595 entries, 88310 to 120443
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        55595 non-null  object 
 1   title         55595 non-null  object 
 2   price         55595 non-null  object 
 3   id            55595 non-null  object 
 4   developer     55595 non-null  object 
 5   release_year  55590 non-null  float64
dtypes: float64(1), object(5)
memory usage: 3.0+ MB


### Normalización de la columna "price".

In [60]:
df_juegos['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 3.99, 18.99, 29.99,
       10.99, 2.99, 1.59, 14.99, 1.99, 59.99, 9.99, 6.99, 7.99, 39.99,
       'Free', 19.99, 7.49, 8.99, 12.99, 5.99, 2.49, 15.99, 1.25, 24.99,
       17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo', 'Play for Free!',
       34.99, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99, 79.99, 49.99,
       13.98, 149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 'Free Mod',
       54.99, 64.99, 74.99, 0.89, 0.5, 'Play Now', 299.99, 1.29, 119.99,
       44.99, 3.0, 15.0, 1.39, 'Free HITMAN™ Holiday Pack', 2.0, 4.0,
       1.95, 1.5, 6.66, 26.99, 399.99, 31.99, 20.0, 40.0, 5.0, 3.33,
       38.85, 71.7, 995.0, 5.49, 27.49, 3.39, 6.0, 19.95, 20.99, 499.99,
       27.99, 199.99, 4.68, 131.4, 44.98, 202.76, 2.3, 0.95, 36.99,
       172.24, 249.99, 2.97, 10.96, 2.66, 6.48, 10.0, 1.0, 11.15,
       'Play the Demo', 49.0, 199.0, 99.0, 87.94, 0.49, 9.98, 9.95, 12.89,
       6.49, 1.87, 

Se observa que hay algunos valores de tipo string cuando solo debe haber de tipo float.

In [61]:
# Se procede a convertir todos los valores no numéricos a precio "0.00".
df_juegos['price'] = pd.to_numeric(df_juegos['price'], errors='coerce')
df_juegos['price'].fillna(0.00, inplace=True)

In [62]:
df_juegos['price'].unique()

array([ 4.99,  0.00,  0.99,  3.99,  18.99,  29.99,  10.99,  2.99,  1.59,
        14.99,  1.99,  59.99,  9.99,  6.99,  7.99,  39.99,  19.99,  7.49,
        8.99,  12.99,  5.99,  2.49,  15.99,  1.25,  24.99,  17.99,  61.99,
        3.49,  11.99,  13.99,  34.99,  1.49,  32.99,  99.99,  14.95,
        69.99,  16.99,  79.99,  49.99,  13.98,  149.99,  771.71,  21.99,
        89.99,  0.98,  139.92,  4.29,  54.99,  64.99,  74.99,  0.89,
        0.50,  299.99,  1.29,  119.99,  44.99,  3.00,  15.00,  1.39,
        2.00,  4.00,  1.95,  1.50,  6.66,  26.99,  399.99,  31.99,  20.00,
        40.00,  5.00,  3.33,  38.85,  71.70,  995.00,  5.49,  27.49,
        3.39,  6.00,  19.95,  20.99,  499.99,  27.99,  199.99,  4.68,
        131.40,  44.98,  202.76,  2.30,  0.95,  36.99,  172.24,  249.99,
        2.97,  10.96,  2.66,  6.48,  10.00,  1.00,  11.15,  49.00,
        199.00,  99.00,  87.94,  0.49,  9.98,  9.95,  12.89,  6.49,  1.87,
        41.99,  4.49,  289.99,  23.96,  5.65,  12.00,  129.99,  23.99

### Carga de datos.

In [63]:
# Se crea un archivo parquet a partir del dataframe.
table = pa.Table.from_pandas(df_juegos)
pq.write_table(table, 'juegos.parquet')